In [3]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("GOOGLE_API_KEY")


In [2]:
from google import genai
from pydantic import BaseModel


# Using a Pydantic model
class Recipe(BaseModel):
    recipe_name: str
    ingredients: list[str]
    total_cost:int

client = genai.Client()
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="List a few popular cookie recipes, and include the amounts of ingredients.",
    config={
        "response_mime_type": "application/json",
        
        #list of dicts (each dict is one Recipe).
        "response_schema": list[Recipe],
        #"propertyOrdering": ["recipeName", "ingredients"]
    },
    
)
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_recipes: list[Recipe] = response.parsed

[{"recipe_name":"Classic Chocolate Chip Cookies","ingredients":["1 cup (2 sticks) unsalted butter, softened","3/4 cup granulated sugar","3/4 cup packed light brown sugar","2 large eggs","1 teaspoon vanilla extract","2 1/4 cups all-purpose flour","1 teaspoon baking soda","1/2 teaspoon salt","1 cup chocolate chips"],"total_cost":10},{"recipe_name":"Simple Sugar Cookies","ingredients":["1 cup (2 sticks) unsalted butter, softened","1 1/2 cups granulated sugar","1 large egg","1 teaspoon vanilla extract","2 1/2 cups all-purpose flour","1 teaspoon baking powder","1/2 teaspoon salt"],"total_cost":8},{"recipe_name":"Chewy Oatmeal Raisin Cookies","ingredients":["1 cup (2 sticks) unsalted butter, softened","1 cup granulated sugar","1 cup packed light brown sugar","2 large eggs","1 teaspoon vanilla extract","1 1/2 cups all-purpose flour","1 teaspoon baking soda","1 teaspoon ground cinnamon","1/2 teaspoon salt","3 cups old-fashioned rolled oats","1 cup raisins"],"total_cost":12}]


In [6]:
# manually construct a schema using google.genai.types.Schema, you can explicitly set fields like propertyOrdering, enum, etc 
from google import genai
from google.genai import types
import pathlib

# Using manual types.Schema
person_schema = types.Schema(
    type="object",
    properties={
        "Doc_type": { "type": "string","enum": ["Gas_safety", "EPC", "EIRC"]},
        "Address": { "type": "string"},
        "Inspection": { "type": "boolean" }
    },
    # mandatory should be displayed
    required=["Doc_type", "Address","Inspection"],   
    # if you  manually construct schema only, we can set property ordering, pydantic can't support this.
    propertyOrdering=["Inspection","Doc_type", "Address"]
)
client = genai.Client()

filepath = pathlib.Path(r"C:\Users\Raghul mani M\Downloads\compliance_docs\compliance_docs\upload_nic_flat_34_old_fire_station_62012f9456696.pdf")

prompt = "extract the document type, address,inspection"
response = client.models.generate_content(
  model="gemini-2.5-flash",
  contents=[
      types.Part.from_bytes(
        data=filepath.read_bytes(),
        # input as pdf
        mime_type='application/pdf',
      ),
      prompt],
    
    config={
        "response_schema": person_schema,
        "response_mime_type": "application/json"
    }
    
    )
print(response.text)

{"Inspection":true,"Doc_type":"EIRC","Address":"OLA FIRE STATIOU GREENICU SE10 OSE"}


In [16]:
# Using list[str] (simple type) response schema structure 
from google import genai
from google.genai import types
import pathlib


client = genai.Client()

filepath = pathlib.Path(r"C:\Users\Raghul mani M\Downloads\compliance_docs\compliance_docs\upload_nic_flat_34_old_fire_station_62012f9456696.pdf")

#just returns a list of strings. No extra structure.
config = {
    #response_mime_type = format (container)
     "response_mime_type": "text/plain", 
    # (or)"response_mime_type": "application/json",
    #response_schema = structure (blueprint of data inside the container)
    #return only one object (one dict or one str one )
    "response_schema": str 
    # (or) "response_schema": dict (Without Pydantic (or a custom types.Schema), you can’t just say dict.Without Pydantic (or a custom types.Schema), we can’t just say dict.)
}

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[types.Part.from_bytes(
        data=filepath.read_bytes(),
        mime_type='application/pdf',
      ),"extract the document type, address,inspection status"],
    config=config
)
print(response.text)


*   **Document Type:** ELECTRICAL INSTALLATION CONDITION REPORT
*   **Address (of the installation):**
    OLA FIRE STATIOU
    GREENICH
    Postcode: SE10 OSE
*   **Inspection Status:** Satisfactory
